In [ ]:
import random
import numpy as np

#Data Ingestor
class DataIngestor:
    def __init__(self, input_data):
        self.raw_data = input_data

    def get_data(self):
        return self.raw_data

# Emission Factor Mapper
class EmissionFactorMapper:
    def __init__(self):
        self.factors = {
            'steel': {'mean': 2.1, 'std': 0.1},
            'transport': {'mean': 0.15, 'std': 0.02}
        }

    def get_factor(self, material):
        return self.factors[material]['mean']

    def get_uncertain_factor(self, material):
        return np.random.normal(
            self.factors[material]['mean'],
            self.factors[material]['std']
        )

# Supply Chain Model
class SupplyChainModel:
    def __init__(self, data):
        self.steel_kg = data.get('steel_kg', 0)
        self.transport_km = data.get('transport_km', 0)

    def get_activities(self):
        return {
            'steel': self.steel_kg,
            'transport': self.transport_km
        }

# Impact Calculator 
class ImpactCalculator:
    def __init__(self, factor_mapper):
        self.mapper = factor_mapper

    def calculate_total(self, activities):
        total = 0.0
        for activity, quantity in activities.items():
            factor = self.mapper.get_factor(activity)
            total += quantity * factor
        return total

# Uncertainty Analyzer
class UncertaintyAnalyzer:
    def __init__(self, supply_model, factor_mapper):
        self.supply_model = supply_model
        self.factor_mapper = factor_mapper

    def monte_carlo(self, n_simulations=1000):
        results = []
        activities = self.supply_model.get_activities()
        for _ in range(n_simulations):
            total = 0.0
            for activity, quantity in activities.items():
                uncertain_factor = self.factor_mapper.get_uncertain_factor(activity)
                total += quantity * uncertain_factor
            results.append(total)
        return np.mean(results), np.std(results), results


In [2]:
# Simulate data input
data = {'steel_kg': 1000, 'transport_km': 500}

# Workflow
ingestor = DataIngestor(data)
factor_mapper = EmissionFactorMapper()
supply_chain = SupplyChainModel(ingestor.get_data())
calculator = ImpactCalculator(factor_mapper)

activities = supply_chain.get_activities()
total_emissions = calculator.calculate_total(activities)
print(f"Point Estimate Emissions: {total_emissions:.2f} tCO2e")

# Run Monte Carlo uncertainty analysis
analyzer = UncertaintyAnalyzer(supply_chain, factor_mapper)
mean_emissions, std_emissions, samples = analyzer.monte_carlo(n_simulations=1000)
print(f"Monte Carlo Mean: {mean_emissions:.2f} tCO2e ± {std_emissions:.2f}")


Point Estimate Emissions: 2175.00 tCO2e
Monte Carlo Mean: 2170.73 tCO2e ± 101.07


We refactored a monolithic LCA script into modular components — each handling a specific task: data ingestion, emission factor mapping, activity modeling, calculation, and uncertainty analysis. Using Monte Carlo simulations, we estimated not just point emissions, but the uncertainty range. In real scenarios, AI can play a huge role: NLP for reading technical reports, GNNs for supply chain modeling, and probabilistic methods for deeper uncertainty understanding.

